In [1]:
print("started script")
from transformers import AutoModelForCausalLM, AutoTokenizer
import torch
print("imported...")

tokenizer = AutoTokenizer.from_pretrained("meta-llama/Llama-2-7b-chat-hf")
model = AutoModelForCausalLM.from_pretrained("meta-llama/Llama-2-7b-chat-hf", device_map = "auto")

print("model loaded!")

started script


imported...


Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

model loaded!


In [12]:
from time import time
max_new_tokens = 400
prompt = "In the era of generative AI, "
medmcqa2 = """
Question.
Low insulin to glucagon ratio is seen in all of these except:
(A) Glycogen synthesis
(B) Glycogen breakdown
(C) Gluconeogenesis
(D) Ketogenesis
Answer.
"""
tokenized_prompt = tokenizer.encode(medmcqa2, return_tensors = "pt").to('cuda')
raw_output = model.generate(tokenized_prompt,
                            max_new_tokens = max_new_tokens,
                            temperature = 0.01)
untokenized_output = tokenizer.decode(raw_output[0], skip_special_tokens = True)
print(untokenized_output)
print(f"output has length {len(raw_output[0])}")
print(f"newly generated {len(raw_output[0]) - len(tokenized_prompt[0])}")


Question.
Low insulin to glucagon ratio is seen in all of these except:
(A) Glycogen synthesis
(B) Glycogen breakdown
(C) Gluconeogenesis
(D) Ketogenesis
Answer.
(C) Gluconeogenesis

Explanation:
A low insulin to glucagon ratio is seen in conditions of insulin resistance, such as type 2 diabetes. In these conditions, the body's cells become less responsive to insulin, leading to high blood glucose levels. Gluconeogenesis is the production of glucose from non-carbohydrate sources, such as lactate, glycerol, and the amino acids. Therefore, option (C) Gluconeogenesis is the correct answer.

Option (A) Glycogen synthesis is the process of forming glycogen from glucose, and is stimulated by insulin. Therefore, a low insulin to glucagon ratio would not lead to an increase in glycogen synthesis.

Option (B) Glycogen breakdown is the process of breaking down glycogen into glucose, and is inhibited by insulin. Therefore, a low insulin to glucagon ratio would not lead to an increase in glycogen

In [ ]:
medmcqa = """
Question:
Axonal transport is:
(A) Antegrade
(B) Retrograde
(C) Antegrade and retrograde
(D) None
Answer: Let's reflect on each response individually, without choosing whether it is correct yet. Then, discuss the responses in relation to the question, hypothesizing on whether it might be the right answer. Finally choose one from these candidates after considering them carefully.
"""